## Setting up the notebook

High-level configs

In [1]:
%reload_ext autoreload
%autoreload 2

from dotenv import load_dotenv

# Load environment variables from .env file. Adjust the path to the .env file as needed.
load_dotenv(dotenv_path='../.env')

# Enable asyncio in Jupyter
import asyncio
import nest_asyncio

nest_asyncio.apply()

#  Add the package to the path (required if you are running this notebook from the examples folder)
import sys
sys.path.append('../../')


Import required packages

In [12]:
import pandas as pd

from lattereview.providers import OpenAIProvider
from lattereview.providers import OllamaProvider
from lattereview.providers import LiteLLMProvider
from lattereview.agents import ScoringReviewer
from lattereview.workflows import ReviewWorkflow

Loading a dummy dataset:

In [3]:
data = pd.read_csv('data.csv')
data

,Title,Abstract,Authors,Year
0,Fusing an agent-based model of mosquito popula...,The mosquito Aedes aegypti is the vector of a ...,"Cavany, S.M.",2022
1,PDRL: Multi-Agent based Reinforcement Learning...,Reinforcement learning has been increasingly a...,"Shaik, T.",2023
2,Learning-accelerated Discovery of Immune-Tumou...,We present an integrated framework for enablin...,"Ozik, J.",2019
3,Investigating spatiotemporal dynamics and sync...,"In this paper we present AceMod, an agent-base...","Cliff, O.M.",2018
4,Modeling the Spread of COVID-19 in University ...,Mathematical and simulation models are often u...,"Herrmann, J.W.",2024
5,Multi-Agent Reinforcement Learning with Action...,Unmanned Aerial Vehicles (UAVs) are increasing...,"Rizvi, D.",2023
6,A new (N) stochastic quenched disorder model f...,Human beings live in a networked world in whic...,"Ferreira, A.A.",2019
7,Frustration induced phases in migrating cell c...,Collective motion of cells is common in many p...,"Copenhagen, K.",2017
8,Universal masking is urgent in the COVID-19 pa...,We present two models for the COVID-19 pandemi...,"Kai, D.",2020
9,Calculus of consent via MARL: Legitimating the...,"Public policies that supply public goods, espe...","Hu, Y.",2021


## Testing the base functionalities

Testing the OpenAI provider (with OpenAI and Gemini models):

In [4]:
openanai_provider = OpenAIProvider(model="gpt-4o-mini")
question = "What is the capital of France?"
asyncio.run(openanai_provider.get_response(question))

('The capital of France is Paris.',
 {'input_cost': 1.05e-06, 'output_cost': 4.2e-06, 'total_cost': 5.25e-06})

Testing the Ollama provider:

In [5]:
ollama_provider = OllamaProvider(model="llama3.2-vision:latest", host="http://localhost:11434")
question = "What is the capital of France?"
asyncio.run(ollama_provider.get_response(question))

('The capital of France is Paris!',
 {'input_cost': 0, 'output_cost': 0, 'total_cost': 0})

Testing the LiteLLM provider:

In [6]:
litellm_provider = LiteLLMProvider(model="gemini/gemini-1.5-flash")

# Other models to try:
# litellm_provider = LiteLLMProvider(model="gpt-4o-mini")
# litellm_provider = LiteLLMProvider(model="claude-3-5-sonnet-20240620")
# litellm_provider = LiteLLMProvider(model="groq/llama-3.3-70b-versatile")
# litellm_provider = LiteLLMProvider(model="ollama/llama3.2-vision:latest")
# litellm_provider = LiteLLMProvider(model="groq/llama-3.3-70b-versatile")

question = "What is the capital of France?"
asyncio.run(litellm_provider.get_response(question))

('The capital of France is Paris.\n', 3e-06)

Testing the ScoringReviewer agent:

In [7]:
agent = ScoringReviewer(
    provider=LiteLLMProvider(model="claude-3-5-sonnet-20241022"),
    name="Pouria",
    max_concurrent_requests=1, 
    backstory="an expert reviewer and researcher!",
    input_description = "article title",
    model_args={"max_tokens": 200, "temperature": 0.1},
    reasoning = "brief",
    scoring_task="Search for articles whose titles suggest they are about AI or machine learning agents.",
    scoring_set=[1, 2],
    scoring_rules='Score 1 if the article title meets the criteria, and 2 if the article paper does not meet the criteria.',
)


# Dummy input
text_list = data.Title.str.lower().tolist()
print("====== Inputs ======\n\n", '\n'.join(text_list[:3]))

# Dummy review
results, total_cost = asyncio.run(agent.review_items(text_list[:3]))
print("\n====== Outputs ======")
for result in results:
    print(result)

# Dummy costs
print("\n====== Costs ======\n")
for i, item in enumerate(agent.memory):
    print(f"Cost for item {i}: {item['cost']}")

print(f"\nTotal cost: {total_cost}")

====== Inputs ======

 fusing an agent-based model of mosquito population dynamics with a statistical reconstruction of spatio-temporal abundance patterns
pdrl: multi-agent based reinforcement learning for predictive monitoring
learning-accelerated discovery of immune-tumour interactions


Reviewing 3 items - 2024-12-24 21:59:43: 100%|██████████| 3/3 [00:05<00:00,  1.82s/it]


====== Outputs ======
{'reasoning': "While this title mentions 'agent-based model', it refers to ecological modeling of mosquito populations rather than AI/ML agents.", 'score': 2, 'certainty': 95}
{'reasoning': "The title explicitly mentions 'multi-agent based reinforcement learning' which directly indicates it deals with AI/machine learning agents", 'score': 1, 'certainty': 100}
{'reasoning': "The term 'learning-accelerated' in the title directly implies the use of machine learning techniques for discovery in immune-tumor interactions.", 'score': 1, 'certainty': 90}

====== Costs ======

Cost for item 0: 0.003318
Cost for item 1: 0.003441
Cost for item 2: 0.003297

Total cost: 0.003297


## Testing the main Functionalities

#### A multiagent review workflow for doing title/abstract analysis

Setting up the agents:

In [8]:
pouria = ScoringReviewer(
    provider=LiteLLMProvider(model="gemini/gemini-1.5-flash"),
    name="Pouria",
    max_concurrent_requests=20, 
    backstory="a radiologist with many years of background in statistcis and data science, who are famous among your colleagues for your systematic thinking, organizaton of thoughts, and being conservative",
    model_args={"max_tokens": 200, "temperature": 0.9},
    reasoning = "brief",
    scoring_task="Look for articles that disucss large languange models-based AI agents applied to medical imaging data",
    scoring_set=[1, 2],
    scoring_rules='Score 1 if the paper meets the criteria, and 2 if the paper does not meet the criteria.',
)

bardia = ScoringReviewer(
    provider=OpenAIProvider(model="gpt-4o-mini"),
    name="Bardia",
    max_concurrent_requests=20, 
    backstory="an expert in data science with a background in developing ML models for healthcare, who are famous among your colleagues for your creativity and out of the box thinking",
    model_args={"max_tokens": 200, "temperature": 0.1},
    reasoning = "brief",
    scoring_task="Look for articles that disucss large languange models-based AI agents applied to medical imaging data",
    scoring_set=[1, 2],
    scoring_rules='Score 1 if the paper meets the criteria, and 2 if the paper does not meet the criteria.',
)

brad = ScoringReviewer(
    provider=OpenAIProvider(model="gpt-4o"),
    name="Brad",
    max_concurrent_requests=20, 
    backstory="a senior radiologist with a PhD in computer science and years of experience as the director of a DL lab focused on developing ML models for radiology and healthcare",
    temperature=0.4,
    reasoning = "cot",
    max_tokens=500,
    scoring_task="""Pouria and Bardia have Looked for articles that disucss large languange models-based AI agents applied to medical imaging data. 
                       They scored an article 1 if they thought it does not meet this criteria, 2 if they thought it meets the criteria, 0 if they were uncertain of scoring.
                       You will receive an article they have had different opinions about, as well as each of their scores and their reasoning for that score. Read their reviews and determine who you agree with. 
                    """,
    scoring_set=[1, 2],
    scoring_rules="""Score 1 if you agree with Pouria, and score 2 if you agree with Bardia.""",
)


Setting up the review workflow:

In [9]:
title_abs_review = ReviewWorkflow(
    workflow_schema=[
        {
            "round": 'A',
            "reviewers": [pouria, bardia],
            "text_inputs": ["Title", "Abstract"]
        },
        {
            "round": 'B',
            "reviewers": [brad],
            "text_inputs": ["Title", "Abstract", "round-A_Pouria_output", "round-A_Bardia_output"],
            "filter": lambda row: row["round-A_Pouria_output"]["score"] != row["round-A_Bardia_output"]["score"]
        }
    ]
)

Applying the review workflow to a number of sample articles:

In [10]:
# Reload the data if needed.
data = pd.read_csv('data.csv')
updated_data = asyncio.run(title_abs_review(data))

print("\n====== Costs ======\n")
print("Total cost: ", title_abs_review.get_total_cost())
print("Detailed costs: ", title_abs_review.reviewer_costs)

updated_data


====== Starting review round A (1/2) ======

Processing 20 eligible rows


['round: A', 'reviewer_name: Pouria'] -                     2024-12-24 21:59:49: 100%|██████████| 20/20 [00:01<00:00, 17.47it/s]


The following columns are present in the dataframe at the end of Pouria's reivew in round A: ['Title', 'Abstract', 'Authors', 'Year', 'round-A_Pouria_output', 'round-A_Pouria_reasoning', 'round-A_Pouria_score', 'round-A_Pouria_certainty']


['round: A', 'reviewer_name: Bardia'] -                     2024-12-24 21:59:50: 100%|██████████| 20/20 [00:03<00:00,  6.55it/s]

The following columns are present in the dataframe at the end of Bardia's reivew in round A: ['Title', 'Abstract', 'Authors', 'Year', 'round-A_Pouria_output', 'round-A_Pouria_reasoning', 'round-A_Pouria_score', 'round-A_Pouria_certainty', 'round-A_Bardia_output', 'round-A_Bardia_reasoning', 'round-A_Bardia_score', 'round-A_Bardia_certainty']

====== Starting review round B (2/2) ======

Skipping review round B - no eligible rows

====== Costs ======

Total cost:  0.00014895
Detailed costs:  {('A', 'Pouria'): 5.324999999999999e-05, ('A', 'Bardia'): 9.57e-05}


,Title,Abstract,Authors,Year,round-A_Pouria_output,round-A_Pouria_reasoning,round-A_Pouria_score,round-A_Pouria_certainty,round-A_Bardia_output,round-A_Bardia_reasoning,round-A_Bardia_score,round-A_Bardia_certainty
0,Fusing an agent-based model of mosquito popula...,The mosquito Aedes aegypti is the vector of a ...,"Cavany, S.M.",2022,"{'certainty': 100, 'reasoning': 'The article d...",The article does not discuss large language mo...,2,100,{'reasoning': 'The article discusses mosquito ...,The article discusses mosquito population dyna...,2,95
1,PDRL: Multi-Agent based Reinforcement Learning...,Reinforcement learning has been increasingly a...,"Shaik, T.",2023,"{'certainty': 100, 'reasoning': 'The article d...",The article does not discuss large language mo...,2,100,{'reasoning': 'The article discusses a reinfor...,The article discusses a reinforcement learning...,2,90
2,Learning-accelerated Discovery of Immune-Tumou...,We present an integrated framework for enablin...,"Ozik, J.",2019,"{'certainty': 100, 'reasoning': 'The abstract ...",The abstract does not mention large language m...,2,100,{'reasoning': 'The article focuses on cancer i...,The article focuses on cancer immunotherapy an...,2,90
3,Investigating spatiotemporal dynamics and sync...,"In this paper we present AceMod, an agent-base...","Cliff, O.M.",2018,"{'certainty': 100, 'reasoning': 'The article d...",The article does not discuss large language mo...,2,100,{'reasoning': 'The article focuses on an agent...,The article focuses on an agent-based modeling...,2,90
4,Modeling the Spread of COVID-19 in University ...,Mathematical and simulation models are often u...,"Herrmann, J.W.",2024,"{'certainty': 100, 'reasoning': 'The article d...",The article does not discuss large language mo...,2,100,{'reasoning': 'The paper focuses on modeling t...,The paper focuses on modeling the spread of CO...,2,95
5,Multi-Agent Reinforcement Learning with Action...,Unmanned Aerial Vehicles (UAVs) are increasing...,"Rizvi, D.",2023,"{'certainty': 100, 'reasoning': 'The article d...",The article does not discuss large language mo...,2,100,{'reasoning': 'The paper discusses multi-agent...,The paper discusses multi-agent reinforcement ...,2,95
6,A new (N) stochastic quenched disorder model f...,Human beings live in a networked world in whic...,"Ferreira, A.A.",2019,"{'certainty': 100, 'reasoning': 'The article d...",The article does not discuss large language mo...,2,100,{'reasoning': 'The article discusses a stochas...,The article discusses a stochastic model for d...,2,90
7,Frustration induced phases in migrating cell c...,Collective motion of cells is common in many p...,"Copenhagen, K.",2017,"{'certainty': 100, 'reasoning': 'The article f...",The article focuses on cell motility and does ...,2,100,{'reasoning': 'The article discusses the motio...,The article discusses the motion of cell clust...,2,90
8,Universal masking is urgent in the COVID-19 pa...,We present two models for the COVID-19 pandemi...,"Kai, D.",2020,"{'certainty': 100, 'reasoning': 'The article f...",The article focuses on epidemiological modelin...,2,100,{'reasoning': 'The article discusses the impac...,The article discusses the impact of universal ...,2,95
9,Calculus of consent via MARL: Legitimating the...,"Public policies that supply public goods, espe...","Hu, Y.",2021,"{'certainty': 100, 'reasoning': 'The article d...",The article does not discuss large language mo...,2,100,{'reasoning': 'The article discusses Multi-Age...,The article discusses Multi-Agent Reinforcemen...,2,95


In [11]:
for i, row in updated_data.iterrows():
    print(
        f"""
        ====== item {i} ======
        Title: {row.Title}
        Abstract: {row.Abstract}
        Pouria's score: {row["round-A_Pouria_score"]}
        Pouria's reasoning: {row["round-A_Pouria_reasoning"]}
        Pouria's certinty: {row["round-A_Pouria_certainty"]}
        Bardia's score: {row["round-A_Bardia_score"]}
        Bardia's reasoning: {row["round-A_Bardia_reasoning"]}
        Bardiad's certainty: {row["round-A_Bardia_certainty"]}
        Brad's score: {None if "round-B_Brad_score" not in row else row["round-B_Brad_score"]}
        Brad's reasoning: {None if "round-B_Brad_reasoning" not in row else row["round-B_Brad_reasoning"]}
        Brad's certainty: {None if "round-B_Brad_certainty" not in row else row["round-B_Brad_certainty"]}
        """
    )


        ====== item 0 ======
        Title: Fusing an agent-based model of mosquito population dynamics with a statistical reconstruction of spatio-temporal abundance patterns
        Abstract: The mosquito Aedes aegypti is the vector of a number of medically-important viruses, including dengue virus, yellow virus, chikungunya virus, and Zika virus, and as such vector control is a key approach to managing the diseases they cause. Understanding the impact of vector control on these diseases is aided by first understanding its impact on Ae. aegypti population dynamics. A number of detail-rich models have been developed to couple the dynamics of the immature and adult stages of Ae. aegypti. The numerous assumptions of these models enable them to realistically characterize impacts of mosquito control, but they also constrain the ability of such models to reproduce empirical patterns that do not conform to the models’ behavior. In contrast, statistical models afford sufficient flexibility 